In [1]:
#include <chrono>
#include "src/tilemap.hpp"
#include "src/player.hpp"
#include "src/menu.hpp"

In [2]:
// gibt die Systemzeit zurück
uint64_t timeSinceEpochMillisec() {
  using namespace std::chrono;
  return duration_cast<milliseconds>(system_clock::now().time_since_epoch()).count();
}

In [ ]:
SVG canvas = SVG(2000, 1500, "Cyber Crawler");
int diff = 0;
int highScore;
int overallKills;
int highestLvl;
std::string key = canvas.lastKey();

// dem getGameStart() ne Referenz auf ne int (int diff) mitliefern, damit main() weiß
// wie schwer das spiel ist

while(true) {
    // Hauptmenue
    Menu *gameMenu = new Menu(highScore, overallKills, highestLvl, &canvas);
    while(!gameMenu -> getGameStart(diff)) {
        key = canvas.lastKey();
        gameMenu -> update(key);
    }
    delete(gameMenu);

    // Dungeon Crawler Spiel
    Rect background = Rect(-100, -100, 2500, 2000, &canvas);
    background.setFill("dimgray");

    TileMap tileMap = TileMap(&canvas);
    EnemyList enemyList = EnemyList(&tileMap, &canvas, diff);
    ItemList itemList = ItemList(&tileMap, &canvas);
    Player player = Player(tileMap.getRandomFreeTile() -> getMapX(), tileMap.getRandomFreeTile() -> getMapY(), &tileMap, &enemyList, &itemList, &canvas);

    uint64_t t = timeSinceEpochMillisec();

    while(!player.getStartNewGame() || (key != "p" && player.getStartNewGame())) {
        key = canvas.lastKey();
        player.keyMovement(key, &enemyList);
        if (t + 1000 <= timeSinceEpochMillisec() && enemyList.getSize() != 0) { 
            for(int i = 0; i < enemyList.getSize(); i++) {
                player.setHP(player.getHP() - enemyList.getEnemy(i) -> autoMove());
            }
            t = timeSinceEpochMillisec();
        }
    }
    itemList.deleteAllImages();
    enemyList.clearEnemyList();
    
    // aktualisiert die Statistiken
    if(player.getScore() > highScore) {
        highScore = player.getScore();
    }
    
    overallKills += player.getKills();
    
    if(player.getLvl() > highestLvl) {
        highestLvl = player.getLvl();
    }
}